In [1]:
# Add python script files to the system env
import sys  
sys.path.insert(1, "..\\My Functions")
# Import the custom python script files
import my_Variogram

In [2]:
my_Variogram.coffee()

KOPO
